<a href="https://colab.research.google.com/github/seenu-g/school_of_ai/blob/master/chap14/DepthMapsTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
import os
folder = '/content/gdrive/My Drive/school_of_ai/chap14/'
os.chdir(folder)
print(os.getcwd()) 
for items in os.listdir(folder):
   print(items)
sys.path.append(folder)

/content/gdrive/My Drive/school_of_ai/chap14
Detectron2 Tutorial.ipynb
collect_data
Background
Foreground
Mask
Overlap_and_mask_Trial.ipynb
DepthMapsTrial.ipynb
DepthModel
Fg-Bg-Mask
Fg-Bg


In [0]:
!git clone https://github.com/seenu-g/DepthModel.git

fatal: destination path 'DepthModel' already exists and is not an empty directory.


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DepthModel/nyu.h5

--2020-05-03 09:38:18--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.102.27
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.102.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DepthModel/nyu.h5’

./DepthModel/nyu.h5 100%[===================>] 164.89M  25.9MB/s    in 9.8s    

2020-05-03 09:38:29 (16.7 MB/s) - ‘./DepthModel/nyu.h5’ saved [172897376/172897376]



In [0]:
ls DepthModel

augment.py      demo_rgb.npy      LICENSE   __pycache__/  train.py
callbacks.py    DenseDepth.ipynb  loss.py   PyTorch/      utils.py
data.py         evaluate.py       model.py  README.md
demo_depth.npy  examples/         nyu.h5    Tensorflow/
demo.py         layers.py         output/   test.py


In [0]:
#!cd DepthModel; python test.py --input "/content/gdrive/My Drive/school_of_ai/chap14/DepthModel/examples/*.png"
!cd DepthModel; python test.py --input "/content/gdrive/My Drive/school_of_ai/chap14/Fg-Bg/fg-bg24000[1-9].jpg" --start 240001 --end 240009
#!cd DepthModel; python test.py --input "/content/gdrive/My Drive/school_of_ai/chap14/Fg-Bg/fg-bg240[0-9][0-9][0-9].jpg" --start 240001 --end 240999 --output "/content/gdrive/My Drive/school_of_ai/chap14/DepthModel/output/"


Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).

Loaded (9) images of size (224, 224, 3).
(112, 112, 3)
(112, 112)
(112, 112)
240002
(112, 112)
(112, 112)
240003
(112, 112)
(112, 112)
240004
(112, 112)
(112, 112)
240005
(112, 112)
(112, 112)
240006
(112, 112)
(112, 112)
240007
(112, 112)
(112, 112)
240008
(112, 112)
(112, 112)
240009
done


In [0]:
from matplotlib import pyplot as plt
from skimage import io

plt.figure(figsize=(20,20))
a = io.imread('/content/gdrive/My Drive/school_of_ai/chap14/DepthModel/output/test240001.jpg')
plt.imshow( io.imread('/content/gdrive/My Drive/school_of_ai/chap14/DepthModel/output/test240001.jpg') )
a.shape

In [0]:
#fg-bg240001.jpg
import glob 
count = 1 ,start = 240 , end = 250
for index in range(start,end):
     for name in glob.glob('/content/gdrive/My Drive/school_of_ai/chap14/Fg-Bg/fg-bg'+ str(index)+'[0-9][0-9][0-9].jpg'): 
      print(count, name)
      count = count + 1 

In [0]:
print(index)

249


In [0]:
ls -l  "/content/gdrive/My Drive/school_of_ai/chap14/DepthModel/output/" ! wc -l

In [0]:
"""
    This function will save the predicted output array as JPEG image under '/content/test'
    Inputs
        outputs: output array 
        input_names_list: list of the filenames of the images
    Output:
        Colormapped JPEG files of depth images generated under '/content/test'
    Can be called after the 'predict' function.
    By default it uses the colormap as 'plasma'
"""

def display_images_new(outputs, input_names_list,output_dir='/content/test/' ):
    is_colormap=True 
    is_rescale=True
    plasma = plt.get_cmap('plasma')
    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    
    all_images = []

    for i in range(outputs.shape[0]):
        imgs = []        
        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
            imgs.append(plasma(rescaled)[:,:,:3])
        else:
            imgs.append(to_multichannel(outputs[i]))

        img_set = np.hstack(imgs)
        #print("Shape:{}".format(np.array(img_set).shape))
        file_name = os.path.join(output_dir,"depth_"+os.path.basename(input_names_list[i]))
        Image.fromarray(np.uint8(np.array(img_set)*255)).save(file_name, 'JPEG',quality=100)
        all_images.append(img_set)

# Helper function to load images from a ZipFile
def load_execute_names(model, zipfile_orig, name_list,output_dir="/content/test"):
    image_list = [Image.open(BytesIO((zipfile_orig.read(name)))) for name in name_list ]   
    inputs = load_images_local(image_list)
    print('Loaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
    outputs = predict(model, inputs)
    converted_img = display_images_new(outputs.copy(),name_list,output_dir=output_dir)
    print("Received output shape:{} converted:{}".format(outputs.shape, converted_img.shape))
    
#### Sample workflow ######

from DenseDepth import utils    #### load_execute_names
from keras.models import load_model
from layers import BilinearUpSampling2D

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'


input_zip = ZipFile("/content/drive/My Drive/EVA4/base_images.zip") ### You can directly read the file from Google-Drive path or from colab. Could't go beyond 300-400 files at a time
file_list = [ name for name in depth_zip.namelist()][:200]

custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}
print('Loading model...')
# Load model into GPU / CPU
model = load_model('/content/nyu.h5', custom_objects=custom_objects, compile=False)
utils.load_execute_names(model, input_zip, file_list)